# Analysis Template

A template with the tools to get started pulling data from a local db.

In [1]:
import os
import asyncio
import requests
import pandas as pd
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
import sqlite3


In [2]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# set up sqlite connection
con = sqlite3.connect("../../data/perps.db")


In [3]:
df_trades = pd.read_sql_query("""
SELECT
    *
FROM
    trades
WHERE
    account = '0xe8c19db00287e3536075114b2576c70773e039bd'
""", con)
df_trades.head(5)

,id,account,abstractAccount,accountType,timestamp,asset,marketKey,orderType,price,margin,positionId,feesPaid,pnl,size,positionSize,positionClosed,trackingCode
0,0x03cd9736876e5dffacb40c7357389204bc82fcad892c...,0xe8c19db00287e3536075114b2576c70773e039bd,0xe8c19db00287e3536075114b2576c70773e039bd,isolated_margin,1679677951,SOL,sSOLPERP,Liquidation,20.960000,0.000000e+00,0x0ea09d97b4084d859328ec4bf8ebcf9ecca26f1d-0x341,1000.000000,-110394.508485,-92513.894791,0.000000,1,0x0000000000000000000000000000000000000000
1,0x0481434f16273b5b113630a437b76028f90eafd767d2...,0xe8c19db00287e3536075114b2576c70773e039bd,0xe8c19db00287e3536075114b2576c70773e039bd,isolated_margin,1677683750,sBTC,sBTCPERP,DelayedOffchain,23551.231252,7.189870e+05,0x59b007e9ea8f89b069c43f8f45834d30853e3699-0x22d,2040.049512,0.000000,-86.536856,-426.166453,0,0x4b57454e544100000000000000000000000000000000...
2,0x085de7b8a2595cb76496070b40ccfb5e81da37dd1cab...,0xe8c19db00287e3536075114b2576c70773e039bd,0xe8c19db00287e3536075114b2576c70773e039bd,isolated_margin,1680047170,sBTC,sBTCPERP,DelayedOffchain,27389.627875,1.358397e+06,0x59b007e9ea8f89b069c43f8f45834d30853e3699-0xb8b,5571.569952,71899.597589,203.345952,0.000000,1,0x4b57454e544100000000000000000000000000000000...
3,0x08ed64b5962bfeb65a3db5aeebddbd43adea99b6fa38...,0xe8c19db00287e3536075114b2576c70773e039bd,0xe8c19db00287e3536075114b2576c70773e039bd,isolated_margin,1679280947,SOL,sSOLPERP,DelayedOffchain,22.583447,2.948872e+05,0x0ea09d97b4084d859328ec4bf8ebcf9ecca26f1d-0x2e6,2409.830863,0.000000,88849.400267,88849.400267,0,0x4b57454e544100000000000000000000000000000000...
4,0x0931e78d48c357b3a79cca3b0dfcb42dc6206e06c5f3...,0xe8c19db00287e3536075114b2576c70773e039bd,0xe8c19db00287e3536075114b2576c70773e039bd,isolated_margin,1678400618,sETH,sETHPERP,DelayedOffchain,1435.775507,1.159395e+06,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0xe94,3497.679647,0.000000,-2434.697924,-8731.601731,0,0x4b57454e544100000000000000000000000000000000...


In [4]:
df_positions = pd.read_sql_query("""
SELECT
    *
FROM
    positions
WHERE
    account = '0xe8c19db00287e3536075114b2576c70773e039bd'
""", con)
df_positions.head(5)

,id,account,abstractAccount,accountType,asset,initialMargin,size,entryPrice,avgEntryPrice,lastPrice,...,closeTimestamp,feesPaid,netFunding,isLiquidated,isOpen,margin,netTransfers,totalDeposits,totalVolume,lastTxHash
0,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x143,0xe8c19db00287e3536075114b2576c70773e039bd,0xe8c19db00287e3536075114b2576c70773e039bd,isolated_margin,MATIC,59998.000000,0,1.209940,1.209940,1.159329,...,1678235395,2577.973872,-45543.937701,0,0,57697.812634,0.00000,0.0,2.144978e+06,0xdcf3f92f0db8701d4e4d898745a2f8146748b8a955ce...
1,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x39f,0xe8c19db00287e3536075114b2576c70773e039bd,0xe8c19db00287e3536075114b2576c70773e039bd,isolated_margin,MATIC,40432.633546,0,1.123701,1.123701,1.098147,...,1679666560,2085.224029,-3665.321508,1,0,0.000000,0.00000,0.0,9.026867e+05,0x56a895c2d3f852a5215c8fc22bd774dc263e4d03596d...
2,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x44e,0xe8c19db00287e3536075114b2576c70773e039bd,0xe8c19db00287e3536075114b2576c70773e039bd,isolated_margin,MATIC,49998.000000,0,1.114758,1.114758,1.127000,...,1680314532,2295.018298,-4513.006110,1,0,0.000000,-12200.98125,0.0,1.096598e+06,0x485c9c138f8a99861d8a68a92d3d2a9880bdcbd5c796...
3,0x0940b0a96c5e1ba33aee331a9f950bb2a6f2fb25-0x1d2,0xe8c19db00287e3536075114b2576c70773e039bd,0xe8c19db00287e3536075114b2576c70773e039bd,isolated_margin,BNB,44998.000000,0,324.822578,324.822578,319.169013,...,1679678528,2691.860985,-4300.406485,0,0,18343.918909,0.00000,0.0,2.239884e+06,0xce83c5193669a5b5e739dafdc356a37b4955415e02b0...
4,0x0940b0a96c5e1ba33aee331a9f950bb2a6f2fb25-0x1f4,0xe8c19db00287e3536075114b2576c70773e039bd,0xe8c19db00287e3536075114b2576c70773e039bd,isolated_margin,BNB,18341.918909,0,325.979293,325.979293,313.628450,...,1680032780,1036.667307,-1888.893227,0,0,32780.545687,0.00000,0.0,8.991280e+05,0x42452d81400f30e1d6fe0f0d26cdcb9b82298f7976dc...


In [5]:
df_transfers = pd.read_sql_query("""
SELECT
    *
FROM
    transfers
WHERE
    account = '0xe8c19db00287e3536075114b2576c70773e039bd' or
    account = '0x7e7f1b10a3061e8ab8c8449ddcc5f95650d9637e'
""", con)
df_transfers.head(5)

,id,account,timestamp,asset,size,market,txHash
0,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x5...,0xe8c19db00287e3536075114b2576c70773e039bd,1678235473,MATIC,-57697.812634,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042,0x5e90470a98723364d38bd15ebbd9406ef27f8df2e174...
1,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x6...,0xe8c19db00287e3536075114b2576c70773e039bd,1677700680,MATIC,60000.000000,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042,0x6f2dd4cbcd0f1478ab9aff879f5e73dde9cb307c86d2...
2,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x9...,0xe8c19db00287e3536075114b2576c70773e039bd,1679518589,MATIC,40434.633546,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042,0x90693da8ed01a4358e0ea1d85886b92c73a44ae23b9e...
3,0x0940b0a96c5e1ba33aee331a9f950bb2a6f2fb25-0x8...,0xe8c19db00287e3536075114b2576c70773e039bd,1679518514,BNB,45000.000000,0x0940b0a96c5e1ba33aee331a9f950bb2a6f2fb25,0x8455540bf509497563446a5322fae2cd0a6a26f0aefd...
4,0x0ea09d97b4084d859328ec4bf8ebcf9ecca26f1d-0x1...,0xe8c19db00287e3536075114b2576c70773e039bd,1679056209,SOL,81701.035650,0x0ea09d97b4084d859328ec4bf8ebcf9ecca26f1d,0x1c34303a5522c92e4fe6913b11e967b881887a900d87...


In [6]:
df_transfers['size'].sum()

5772762.047287755

In [7]:
df_trades['pnl'].sum()

-2676564.0671981173

In [8]:
df_positions['pnl'].sum()

-2676564.0671981177

In [9]:
df_positions['netFunding'].sum()

-2023592.8559527127

In [10]:
df_positions['feesPaid'].sum()

1072573.1241369236